In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [3]:
# 관측소 코드 담기

where = 72501   # 72501 OKX Upton Observations

In [36]:
where = 74389 # GYX Gray Observations

In [54]:
where = 72403 # IAD Sterling Observations

In [60]:
final = pd.DataFrame()

# 30일, 31일 월 분리
# 2월은 따로 생각해야 함
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2008, 2021):   # 여기 바꾸기
    for month in range(1, 13): 
        if month in months_30: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3012&STNM={}'.format(year, month, where), verify=False)
        elif month in months_31: 
            response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=3112&STNM={}'.format(year, month, where), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2912&STNM={}'.format(year, month, where), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={}&MONTH={}&FROM=0100&TO=2812&STNM={}'.format(year, month, where), verify=False)
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        title = soup.select('body > h2')  # title은 하나씩 있음
        data = soup.select('body > pre')  # pre는 하나 건너 하나씩 concat 해야함

        idx_data = 0
        for idx in range(len(title)): 
            data_df = pd.DataFrame(data[idx_data])
            data_df[0] = data_df[0].str.replace("-----------------------------------------------------------------------------", "")
            data_df_2 = data_df[0].str.split('\n', expand=True)

            data_df_2.drop([0, 1, 2, 3, 4], axis = 1, inplace = True)

            data_df_2 = data_df_2.transpose()
            title_tmp = str(title[idx])
            data_df_2['date'] = title_tmp[4:54]   # upton 기준 [4:51] # gray 기준 [4:50] # Sterling 기준 [4:54]
            
            final = pd.concat([final, data_df_2], axis = 0)

            idx_data += 2

c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.uwyo.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\urllib3\connecti

In [59]:
# sterling 08-20 돌리는 중
# 08-15, 16-20 나눠서 파일 만들자

title_tmp[4:54]

'72403 IAD Sterling Observations at 12Z 22 Jan 2008'

In [61]:
final

,0,date
5,1008.0 93 11.8 -0.2 44 3.76 ...,72403 IAD Sterling Observations at 00Z 01 May ...
6,1006.0 108 13.6 -0.4 38 3.71 ...,72403 IAD Sterling Observations at 00Z 01 May ...
7,1000.0 154 13.8 -0.2 38 3.79 ...,72403 IAD Sterling Observations at 00Z 01 May ...
8,982.0 305 12.4 -0.9 40 3.66 ...,72403 IAD Sterling Observations at 00Z 01 May ...
9,946.8 610 9.6 -2.3 43 3.42 ...,72403 IAD Sterling Observations at 00Z 01 May ...
...,...,...
112,7.8 32032 -37.3 -79.3 0 0.10 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
113,7.5 32309 -48.2 -82.3 1 0.06 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
114,7.4 32374 -50.7 -83.0 1 0.06 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
115,7.0 32735 -64.9 -86.9 3 0.03 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...


In [62]:
final2 = final.reset_index(drop = True)
final2

,0,date
0,1008.0 93 11.8 -0.2 44 3.76 ...,72403 IAD Sterling Observations at 00Z 01 May ...
1,1006.0 108 13.6 -0.4 38 3.71 ...,72403 IAD Sterling Observations at 00Z 01 May ...
2,1000.0 154 13.8 -0.2 38 3.79 ...,72403 IAD Sterling Observations at 00Z 01 May ...
3,982.0 305 12.4 -0.9 40 3.66 ...,72403 IAD Sterling Observations at 00Z 01 May ...
4,946.8 610 9.6 -2.3 43 3.42 ...,72403 IAD Sterling Observations at 00Z 01 May ...
...,...,...
780891,7.8 32032 -37.3 -79.3 0 0.10 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
780892,7.5 32309 -48.2 -82.3 1 0.06 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
780893,7.4 32374 -50.7 -83.0 1 0.06 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
780894,7.0 32735 -64.9 -86.9 3 0.03 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...


In [66]:
final2['date'].str[46:]

0         2008
1         2008
2         2008
3         2008
4         2008
          ... 
780891    2020
780892    2020
780893    2020
780894    2020
780895    2020
Name: date, Length: 780896, dtype: object

In [67]:
final3 = final2.copy()
final2_0817 = final3.loc[final3['date'].str[46:] <= '2017'].reset_index(drop = True)
final2_1820 = final3.loc[final3['date'].str[46:] > '2017'].reset_index(drop = True)

In [68]:
final2_1820

,0,date
0,1018.0 93 -8.5 -14.5 62 1.23 ...,72403 IAD Sterling Observations at 00Z 01 Jan ...
1,1000.0 230 -9.5 -16.5 57 1.06 ...,72403 IAD Sterling Observations at 00Z 01 Jan ...
2,990.3 305 -10.2 -16.6 60 1.06 ...,72403 IAD Sterling Observations at 00Z 01 Jan ...
3,959.0 552 -12.5 -16.8 70 1.08 ...,72403 IAD Sterling Observations at 00Z 01 Jan ...
4,951.7 610 -13.0 -17.4 70 1.03 ...,72403 IAD Sterling Observations at 00Z 01 Jan ...
...,...,...
141046,7.8 32032 -37.3 -79.3 0 0.10 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
141047,7.5 32309 -48.2 -82.3 1 0.06 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
141048,7.4 32374 -50.7 -83.0 1 0.06 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...
141049,7.0 32735 -64.9 -86.9 3 0.03 ...,72403 IAD Sterling Observations at 12Z 31 Dec ...


In [16]:
final2.to_csv('D:\\SNUlab\\0. data\\thermal inversion\\Upton_crawling_0820.csv', encoding = 'euc-kr', index = False)

In [69]:
final2_0817.to_csv('D:\\SNUlab\\0. data\\thermal inversion\\Sterling_crawling_0817.csv', encoding = 'euc-kr', index = False)
final2_1820.to_csv('D:\\SNUlab\\0. data\\thermal inversion\\Sterling_crawling_1820.csv', encoding = 'euc-kr', index = False)